<a href="https://colab.research.google.com/github/DiegoJSG/Spark_ferramenta_e_suas_funcionalidades/blob/main/Spark(conhecendo_a_ferramenta).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências toda vez que encerrar a sessao e comecar uma nova
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [8]:
#configurar variaveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [9]:
# importando e iniciando o findspark
import findspark
findspark.init()

In [ ]:
# exemplo cas queira entender melhor como o spark UI funciona
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .master('local[*]') \
#     .appName("Iniciando com Spark") \
#     .getOrCreate()

In [ ]:
# apertando (ctrl+/ ) comenta a sessao de codigo (#) e (crtl +k+u) descomenta voltando ao estao normal

# spark

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
     .master('local[*]')\
    .appName("Iniciando com Spark")\
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [11]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [12]:
get_ipython().system_raw('./ngrok authtoken 2RTL6KWgi7y28itU58wf17nxQgm_34S2snMgoq7se25A9Cjeu')
get_ipython().system_raw('./ngrok http 4050 &')

In [13]:
#apos executar a cedula acima espere alguns segundos para executar o !CURL abaixo
#e sempre que executal ele dara uma "public_url" nova
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://882c-34-73-182-140.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://882c-34-73-182-140.ngrok-free.app","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [9]:
#montado um drive exemplo
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#apenas exemplo nao precisa executar
import zipfile
zipfile.ZipFile('/content/drive/MyDrive/ARQUIVOASERIMPORTADO.zip', 'r').extractall('/content/drive/MyDrive/PASTAEXEMPLO/')

In [ ]:
#exemplo de path para uma pasta com arquivos .CSV
path = '/content/drive/MyDrive/PASTAEXEMPLO/empresas'
empresas = spark.read.csv(path, sep=';', inferSchema=True)

#path = '/content/drive/MyDrive/PASTAEXEMPLO/empresas/part-0004-547.csv' ou /part-000*
#um exemplo caso queira um arquivo especifico e nao a pasta toda

In [ ]:
#contaria o total empresas do arquivo
empresas.count()

In [ ]:
empresas.limit(5).toPandas()
#mostra um limite de 5 empresas

In [ ]:
#mostrara em lista o indice dos itens do empresasColNames
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

for index, ColName in enumarete(empresasColNames):
    empresas = empresas.withColumnRenamed(f"_c{index}", colName)
    #c = colunas
empresas.columns
# iria mudar o nome das colunas (c1,c2,c3) para cada um dos intens listao no empresasColNames
# ao executar novamente empresas.limit(5).toPandas()

In [ ]:
empresas.printSchema()
#mostra o tipo e colunas com oque esta trabalhando no caso aqui EMPRESAS

In [ ]:
#trasformar uma string en double
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [ ]:
# função de transformação do dataframe utilizada para alterar valores e tipos de dados
empresas = empresas.withColumn('capital_social_da_empresa', f.regexp_replace('capital_social_da_empresa', ',', '.'))
# no caso toda vez que encontrar ',' (virgula) , '.' sera subistituido por (ponto)
empresas.limit(5).toPandas()
# execute novamente para ver o resultado

In [ ]:
# trasformando uma string em double no caso na coluna (capital_social_da_empresa)
empresas = empresas.withColumn('capital_social_da_empresa', empresas['capital_social_da_empresa'].cast(DoubleType()))
empresas.printSchema()
# para ver o resultado

In [ ]:
#([(20200924,)] simboliza um data 2020/09/04 - yyyyMMdd
df = spark.createDataFrame([(20200924,), (20201022,), (20210215,)], ['data'])
df.toPandas()

In [ ]:
df = df.withColumn("data", f.to_date(df.data.cast(StringType()), 'yyyyMMdd'))
df.printSchema()
#sendo o resultado exemplo 2020-09-24

In [ ]:
#exemplos utilizando o parametro criado acima o (\) indica pular uma linha
estabelecimentos = estabelecimentos\
            .withColumn(
            "data_situacao_cadastral",
            f.to_date(estabelecimentos.data_situacao_cadastral.cast(StringType()), 'yyyyMMdd')
            )\
            .withColumn(
            "data_de_inicio_atividade",
            f.to_date(estabelecimentos.data_de_inicio_atividade.cast(StringType()), 'yyyyMMdd')
            )\
            .withColumn(
            "data_da_situacao_especial",
            f.to_date(estabelecimentos.data_da_situacao_especial.cast(StringType()), 'yyyyMMdd')
            )\

In [ ]:
empresas\
        .select('*')\
        .show(5)
# mostra tudo na seleção empresas até o quinto (5) item e se colocar (5, False)
# nomes muitos extensos que as vezes ficariam com ... seria totalmente exibidos

In [ ]:
empresas\
        .select('natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa')\
        .show(5)
# mostra apenas as ares selecionadas entre ()

In [ ]:
df = spark.createDataFrame([(1, ), (2, ), (3, ), (None, )], ['data'])
df.toPandas()
df.show()
#a informação final aparecera Nulll (quando se usa o None)

In [ ]:
df = spark.createDataFrame([(1.,), (2.,), (3.,), (float('nan'), )], ['data'])
df.toPandas()
df.show()
#a informação final aparecera NaN (quando se usa o float)

In [ ]:
# função (f) de contagem (count) exemplo
socios.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in socios.columns]).show()

In [ ]:
#Filtrando resultados
# codigo com uma string com uma (condição)
empresas\
            .where("capital_social_da_empresa==50)\
            .show(5, False)

In [ ]:
# mais "detalhada" por fora de uma condição de uma string
socios\
        .select("nome_do_socio_ou_razao_social")\
        .filter(socios.nome_do_socio_ou_razao_social)\
        .filter(socios.nome_do_socio_ou_razao_social)\

In [ ]:
# ou essa com um nome como exemplo e com o limite de busca no caso (10)
socios\
        .select("nome_do_socio_ou_razao_social")\
        .filter(socios.nome_do_socio_ou_razao_social.startwith("RODRIGO"))\
        .filter(socios.nome_do_socio_ou_razao_social.endswith("DIAS"))\
        .limit(10)\
        .toPandas()

In [ ]:
# outro exemplo para criar funcao
df = spark.createDataFrame([('RESTAURANTE DO RUI',), ('Juca restaurantes ltda',), ('Joca Restaurante',)], ['data'])
df.toPandas()

In [ ]:
# .upper procura letras maiuscula no caso a palavra RESTAURANTE
# e se estivesse 'RESTAURANTE' sem o (%%) só retornaria o RESTAURANTE DO RUI
# o % na frente procura pela palavra na frente da palavra %RESTAURANTE no caso (Joca Restaurante)
# e o % depois de restaurante seria tudo depois da palavra restaurante RESTAURANTE%
df\
    .where(f.upper(df.data).like('%RESTAURANTE%'))\
    .show(truncate=False)

In [ ]:
#outro exemplo com mais exemplo para busca
empresas\
    .select('razao_social_nome_empresarial', 'natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa')\
    .filter(f.upper(empresas['razao_social_nome_empresarial']).like('%RESTAURANTE%'))\
    .show(15, False)